<a href="https://colab.research.google.com/github/jorgemunozl/vla-test/blob/main/sixth_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Six Test - Simulate Inference

Inference with the subtask code implementation, the text is now the most reasonable but it has some sense. The only issue is that the COMPLEMENTARY DATA is on blank. But the task is filling it nice. So there exist probability that another probability that we are calling the wrong COMPLEMENTARY DATA.

In [1]:
import os

# Set this BEFORE importing pytorch/tensorflow
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import torch
# Check if it worked (should return 1 if you selected a single GPU)
print(torch.cuda.device_count()) 

1


In [2]:
DS_ID = "NONHUMAN-RESEARCH/test-general-idx"
PRETRAINED_PATH = "NONHUMAN-RESEARCH/vlm_training_fruits"

In [3]:
from xhuman.policies.pi05ki.configuration_pi05ki import PI05KIConfig

policy_config = PI05KIConfig(repo_id="none",device="cuda",pretrained_path=PRETRAINED_PATH)

In [4]:
from xhuman.configs.default import LerobotDatasetConfig

dataset_config = LerobotDatasetConfig(
    repo_id=DS_ID,
)

In [18]:
from xhuman.configs.train import TrainPipelineConfigXHUMAN

cfg = TrainPipelineConfigXHUMAN(
    dataset=dataset_config,
    policy=policy_config,
)
cfg.dataset.train_with_subtask = True
cfg.validate()

In [19]:
from xhuman.datasets.factory import make_dataset_xhuman

dataset = make_dataset_xhuman(cfg)


Failed to read file '/home/lperez/.cache/huggingface/lerobot/NONHUMAN-RESEARCH/test-general-idx/data/chunk-000/file-000.parquet' with error <class 'datasets.table.CastError'>: Couldn't cast
action: list<element: float>
  child 0, element: float
observation.state: list<element: float>
  child 0, element: float
timestamp: float
frame_index: int64
episode_index: int64
index: int64
task_index: int64
general_task_index: int64
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 1051
to
{'action': List(Value('float32'), length=14), 'observation.state': List(Value('float32'), length=14), 'timestamp': Value('float32'), 'frame_index': Value('int64'), 'episode_index': Value('int64'), 'index': Value('int64'), 'task_index': Value('int64')}
because column names don't match


[10:15:10] ERROR    Failed to read file                                                              ]8;id=499115;file:///home/lperez/miniconda3/envs/xhuman/lib/python3.12/site-packages/datasets/packaged_modules/parquet/parquet.py\parquet.py]8;;\:]8;id=636943;file:///home/lperez/miniconda3/envs/xhuman/lib/python3.12/site-packages/datasets/packaged_modules/parquet/parquet.py#108\108]8;;\
                    '/home/lperez/.cache/huggingface/lerobot/NONHUMAN-RESEARCH/test-general-idx/data               
                    /chunk-000/file-000.parquet' with error <class 'datasets.table.CastError'>:                    
                    Couldn't cast                                                                                  
                    action: list<element: float>                                                                   
                      child 0, element: float                                                                      
                    observation.state: list<element: float>                                                        
                      child 0, element: float                                                                      
                    timestamp: float                                                                               
                    frame_index: int64                                                                             
                    episode_index: int64                                                                           
                    index: int64                                                                                   
                    task_index: int64                                                                              
                    general_task_index: int64                                                                      
                    -- schema metadata --                                                                          
                    pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' +                   
                    1051                                                                                           
                    to                                                                                             
                    {'action': List(Value('float32'), length=14), 'observation.state':                             
                    List(Value('float32'), length=14), 'timestamp': Value('float32'), 'frame_index':               
                    Value('int64'), 'episode_index': Value('int64'), 'index': Value('int64'),                      
                    'task_index': Value('int64')}                                                                  
                    because column names don't match                                                               

In [21]:
dataset.train_with_subtask

AttributeError: 'XHumanDataset' object has no attribute 'train_with_subtask'

In [7]:
from xhuman.policies.factory import make_xhuman_policy

policy = make_xhuman_policy(
    cfg=cfg.policy,
    ds_meta=dataset.meta,
)

Loading model from: NONHUMAN-RESEARCH/vlm_training_fruits
✓ Loaded state dict from model.safetensors


[10:12:33] WARNING  Vision embedding key might need handling:                               ]8;id=255821;file:///home/lperez/main/nh/XHUMAN/xhuman/policies/pi05ki/modeling_pi05ki.py\modeling_pi05ki.py]8;;\:]8;id=149120;file:///home/lperez/main/nh/XHUMAN/xhuman/policies/pi05ki/modeling_pi05ki.py#1832\1832]8;;\
                    model.paligemma_with_expert.paligemma.model.vision_tower.vision_model.e                        
                    mbeddings.patch_embedding.bias                                                                 

           WARNING  Vision embedding key might need handling:                               ]8;id=479091;file:///home/lperez/main/nh/XHUMAN/xhuman/policies/pi05ki/modeling_pi05ki.py\modeling_pi05ki.py]8;;\:]8;id=389066;file:///home/lperez/main/nh/XHUMAN/xhuman/policies/pi05ki/modeling_pi05ki.py#1832\1832]8;;\
                    model.paligemma_with_expert.paligemma.model.vision_tower.vision_model.e                        
                    mbeddings.patch_embedding.weight                                                               

	Missing key(s) in state_dict: "model.paligemma_with_expert.paligemma.model.language_model.embed_tokens.weight". 


In [8]:
from xhuman.policies.factory import make_xhuman_pre_post_processors

preprocessor, _ = make_xhuman_pre_post_processors(
        policy_cfg=policy_config,
        pretrained_path=policy_config.pretrained_path,
    )

In [9]:
device = torch.device("cuda")

In [10]:
    processor_kwargs = {}
    postprocessor_kwargs = {}
    if (cfg.policy.pretrained_path and not cfg.resume) or not cfg.policy.pretrained_path:
        # Only provide dataset_stats when not resuming from saved processor state
        processor_kwargs["dataset_stats"] = dataset.meta.stats

    if cfg.policy.pretrained_path is not None:
        print("here")
        processor_kwargs["preprocessor_overrides"] = {
            "device_processor": {"device": device.type},
            "normalizer_processor": {
                "stats": dataset.meta.stats,
                "features": {**policy.config.input_features, **policy.config.output_features},
                "norm_map": policy.config.normalization_mapping,
            },
        }
        postprocessor_kwargs["postprocessor_overrides"] = {
            "unnormalizer_processor": {
                "stats": dataset.meta.stats,
                "features": policy.config.output_features,
                "norm_map": policy.config.normalization_mapping,
            },
        }

    preprocessor, postprocessor = make_xhuman_pre_post_processors(
        policy_cfg=cfg.policy,
        pretrained_path=cfg.policy.pretrained_path,
        **processor_kwargs,
        **postprocessor_kwargs,
    )

here


In [11]:
preprocessor

DataProcessorPipeline(name='policy_preprocessor', steps=6: [RenameObservationsProcessorStep, AddBatchDimensionProcessorStep, ..., DeviceProcessorStep])

In [17]:
frame = dataset[0]
frame.keys()

dict_keys(['observation.images.left', 'observation.images.top', 'observation.images.right', 'action', 'observation.state', 'timestamp', 'frame_index', 'episode_index', 'index', 'task_index', 'general_task_index', 'action_is_pad', 'task'])

In [14]:
batch_subtask = preprocessor.subtask(frame)

In [16]:
batch_subtask.keys()

dict_keys(['action', 'next.reward', 'next.done', 'next.truncated', 'info', 'action_is_pad', 'task', 'index', 'task_index', 'episode_index', 'observation.images.left', 'observation.images.top', 'observation.images.right', 'observation.state', 'observation.language.tokens', 'observation.language.attention_mask'])

In [ ]:
img, img_mask = policy._preprocess_images(batch_subtask)

NameError: name 'batch_subtask' is not defined

In [ ]:
tokens , mask = batch_subtask["observation.language.tokens"], batch_subtask["observation.language.attention_mask"]
# img, batch_subtask["observation.images.top"]

In [ ]:
policy._detokenize_subtask(tokens)

In [ ]:
mask_ai = img_mask[0]
mask_ai

In [ ]:
import torch

In [ ]:
mask_2 = [fal, mask_ai, fal]

In [ ]:
for i in range(100):
    frame = change_task(dataset[i])
    time_index = frame["frame_index"]

    # Add the cached subtask to the frame BEFORE preprocessing
    if policy.cached_subtask is not None:
        frame["subtask"] = policy.cached_subtask

    if time_index % policy_config.subtask_prediction_frequency == 0:
        print("Time Index", time_index.item())
        batch_subtask = preprocessor.subtask(frame)
        print("Batch Task", batch_subtask["task"][0])
        policy.update_subtask(batch_subtask)
        print("Policy cached_subtask:", policy.cached_subtask)

        # Update frame with new subtask for next iteration
        frame["subtask"] = policy.cached_subtask

    batch = preprocessor(frame)


    if i % 50 == 0:
        print("\n" + "="*80)
        print(f"Step {i} | Time Index: {time_index}")
        print("-"*80)
        print(f"Batch Task (prompt): {batch['task'][0]}")
        print("="*80 + "\n")

Time Index 0
Batch Task Task: put the fruits in the basket. Subtask: 
Policy cached_subtask: left뀜⡯

Step 0 | Time Index: 0
Batch Task (prompt): Task: left뀜⡯. State: -1 0 255 114 77 112 -1 164 -1 255 175 53 20 2 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128;
Action: 


Step 50 | Time Index: 50
Batch Task (prompt): Task: left뀜⡯. State: 7 0 255 101 57 167 -1 212 0 255 164 71 47 0 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128;
Action: 

Time Index 100
Batch Task Task: put the fruits in the basket. Subtask: 
Policy cached_subtask: , the⸂

Step 100 | Time Index: 100
Batch Task (prompt): Task: , the⸂. State: 7 0 255 102 60 160 -1 211 0 255 164 71 47 0 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128;
Action: 


Step 150 | Time Index: 150
Batch Task (prompt): Task: , the⸂. State: 73 42 184 124 151 167 42 161 75 168 163 172 5 66 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128;
Action: 

Time Index 200
Batch Task Task: put the fruits in the basket. Subtask: 
Policy cached_subtask: left舠

Step 200 | Time Index: 200
Batch Task (prompt): Task: left舠. State: 52 50 188 140 128 148 48 60 159 147 255 223 118 255 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128;
Action: 


Step 250 | Time Index: 250
Batch Task (prompt): Task: left舠. State: 8 52 183 160 139 134 45 24 210 157 255 157 169 255 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128;
Action: 

Time Index 300
Batch Task Task: put the fruits in the basket. Subtask: 
Policy cached_subtask: left

Step 300 | Time Index: 300
Batch Task (prompt): Task: left. State: 13 45 193 163 131 125 39 48 198 113 255 199 170 198 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128 128;
Action: 
